# Combining Datasets - Intro

In this chapter, you will learn how to combine data from different sources. This mostly involves a so called **join** (or, sometimes, "merge") of tables with different columns. In that case, rows from one table are *matched* using columns from both tables that contain common "identifiers" (e.g. a compy ID or name, a period indicator such as *year* etc.).

In addition, we will cover the case of a **union** (or, sometimes, "append" or "concat(enation)" operation) where the rows of one table are *added* to the rows of another table. These tables typically have the *same* columns (or at least they share a common subset of columns).

# Preparations

In [1]:
import pandas as pd

pd.set_option("display.max_columns", 500)

# Joins (`pd.DataFrame.merge()`)

## Overview

When two tables are combined by matching their rows this is most commonly called a *join*, especially in the context of relational databases and SQL. In some programming languages, including *Python*, *R*, and *Stata* this operation is also called a *merge*.

For example, we might want to merge a dataset containing time-invariant data on countries (e.g. region or currency) with time-variant data like population or GDP.

When two tables are *joined*, it is common to call one table the *'left'* table and the other the *'right'* table. Accordingly, there are **four types of joins** :
1. ) **inner join**: keep rows that have matching values in both tables.

2. ) **left join**: keep all rows from the left table, and the matched rows from the right table.

3. ) **right join**: keep all rows from the right table, and the matched rows from the left table.

4. ) **full join**: keep all rows from both tables.

When matching rows from one table to another, there can be **three different relations** between the rows from both tables:

1. ) **1:1**

    * One row in Table A matches one row in Table B OR there is no match.
    * e.g. two tables containing different time-invariant data on countries

2. ) **1:m**

	* One row in Table A matches one or more row in Table B OR none at all.
    * e.g. the aforementioned example of Table A containing time-invariant and Table B containing data for several years per country.

3. ) **m:m**

    * One or more rows in Table A match one or more rows in Table B OR there is no match at all.
    * e.g. Firms can belong to several industries and industries typically contain several firms.
    * Note: direct m:m merges are somewhat rare in practice. Typically, one has to perform two or more 1:m or n:1 merges in sequence, instead.
    * The special case where *each* row of one table is matched with *each* row of the other table. This is called a **cross-join**.

See the file **"join_examples.xlsx"** in the raw-data folder for illustrated examples.

## The `pd.DataFrame.merge()` method

The `pd.DataFrame.merge()` method can be used to perform database-style joins.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on.

In the following, we will see the typical join situations in action.

Also see [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) and [https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#merging-join](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#merging-join)

## 1:1 joins

### Loading the two tables

In [2]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_11_Table_A")
df_left.head(10)

countryname countrycode
0         Germany         DEU
1          France         FRA
2  United Kingdom         GBR
3            Iraq         IRQ
4   United States         USA

In [3]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_11_Table_B")
df_right.head(10)

countrycode    currencyunit          incomegroup
0         ASM     U.S. dollar  Upper middle income
1         BWA   Botswana pula                  NaN
2         DEU            Euro          High income
3         FRA            Euro          High income
4         GBR  Pound sterling          High income

### Inner join

`how` and `on` define the kind of join and the common identifiers.

In [4]:
df_join = df_left.merge(df_right, how="inner", on="countrycode")
df_join

countryname countrycode    currencyunit  incomegroup
0         Germany         DEU            Euro  High income
1          France         FRA            Euro  High income
2  United Kingdom         GBR  Pound sterling  High income

It  is good practice to validate the integrity of the assumed relation between the tables' rows, here '1:1'. `.merge()` can do this by passing the appropriate `validate` argument (note, that the 1:1 relation is a special case of the 1:m relation!):

In [5]:
# explicitly expect a 1:1 relation
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:1")

# expect a 1:m relation instead:
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:m")

In [6]:
# Let's force an error by duplicating the left table
df_left_times_two = pd.concat([df_left, df_left])
try:
    df_left_times_two.merge(df_right, how="inner", on="countrycode", validate="1:1")
except Exception as e:  # noqa: BLE001
    print(f"Error: {e}")

Error: Merge keys are not unique in left dataset; not a one-to-one merge


You should also check which rows from the two tables are **not matched**:

In [7]:
df_left.loc[~df_left["countrycode"].isin(df_join["countrycode"])]

countryname countrycode
3           Iraq         IRQ
4  United States         USA

In [8]:
df_right.loc[~df_right["countrycode"].isin(df_join["countrycode"])]

countrycode   currencyunit          incomegroup
0         ASM    U.S. dollar  Upper middle income
1         BWA  Botswana pula                  NaN

### Left join

In [9]:
df_join = df_left.merge(df_right, how="left", on="countrycode", validate="1:1")
df_join

countryname countrycode    currencyunit  incomegroup
0         Germany         DEU            Euro  High income
1          France         FRA            Euro  High income
2  United Kingdom         GBR  Pound sterling  High income
3            Iraq         IRQ             NaN          NaN
4   United States         USA             NaN          NaN

In [10]:
# still missing a few rows from the right
df_right.loc[~df_right["countrycode"].isin(df_join["countrycode"])]

countrycode   currencyunit          incomegroup
0         ASM    U.S. dollar  Upper middle income
1         BWA  Botswana pula                  NaN

### Right join

In [11]:
df_join = df_left.merge(
    df_right, how="right", on="countrycode", validate="1:1", indicator="check_merge"
)
df_join

countryname countrycode    currencyunit          incomegroup check_merge
0             NaN         ASM     U.S. dollar  Upper middle income  right_only
1             NaN         BWA   Botswana pula                  NaN  right_only
2         Germany         DEU            Euro          High income        both
3          France         FRA            Euro          High income        both
4  United Kingdom         GBR  Pound sterling          High income        both

In [12]:
df_left.loc[~df_left["countrycode"].isin(df_join["countrycode"])]

countryname countrycode
3           Iraq         IRQ
4  United States         USA

### Outer join

The outer join is very useful because it is easier to see which rows have not been matched - a fact that you should **always be aware** of! Therefore, in practice, it is recommended to start with the outer join and then drop rows in a *controlled* fashion according to the desired final result. For example, here we start with an outer join but reduce the final result to an inner join, explicitly taking account of the dropped rows on the way:

In [13]:
df_join = df_left.merge(df_right, how="outer", on="countrycode", validate="1:1")
df_join

countryname countrycode    currencyunit          incomegroup
0             NaN         ASM     U.S. dollar  Upper middle income
1             NaN         BWA   Botswana pula                  NaN
2         Germany         DEU            Euro          High income
3          France         FRA            Euro          High income
4  United Kingdom         GBR  Pound sterling          High income
5            Iraq         IRQ             NaN                  NaN
6   United States         USA             NaN                  NaN

In [14]:
# Validate that we have not lost any rows
# this time, we make use of the `indicator` option:
df_join = df_left.merge(
    df_right, how="outer", on="countrycode", validate="1:1", indicator="check_merge"
)
df_join["check_merge"].value_counts()

check_merge
both          3
left_only     2
right_only    2
Name: count, dtype: int64

In [15]:
# Rows not matched from the left table
df_join[df_join["check_merge"] == "left_only"]

countryname countrycode currencyunit incomegroup check_merge
5           Iraq         IRQ          NaN         NaN   left_only
6  United States         USA          NaN         NaN   left_only

In [16]:
# Rows not matched from the right table
df_join[df_join["check_merge"] == "right_only"]

countryname countrycode   currencyunit          incomegroup check_merge
0         NaN         ASM    U.S. dollar  Upper middle income  right_only
1         NaN         BWA  Botswana pula                  NaN  right_only

In [17]:
# Rows equivalent to inner join
df_join[df_join["check_merge"] == "both"]

countryname countrycode    currencyunit  incomegroup check_merge
2         Germany         DEU            Euro  High income        both
3          France         FRA            Euro  High income        both
4  United Kingdom         GBR  Pound sterling  High income        both

## Exercise 1

1. Load the first sheet of the Excel file "wdi_reduced.xlsx" into a pandas DataFrame (see [here](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for help with `pandas.read_excel()`)

2. Load the annual CO2 emmissions per capita from the file "wdi_EN_ATM_CO2E_PC.csv".

3. (Inner) join the CO2 emmissions with our main dataset from "wdi_reduced". Make sure to validate the expected 1:1 relation and to check for any unmatched rows from both tables.

4. BONUS: Load the annual unemployment rates from the file "wdi_SL_UEM_TOTL_ZS.xlsx".

5. BONUS: Reshape the annual unemployment rates dataset to long format containg one row per *country* and *year*.

6. BONUS: (Inner) join it with our main dataset from "wdi_reduced". Make sure to validate the expected 1:1 relation and to check for any unmatched rows from both tables.

## 1:m joins

### Loading the two tables

In [18]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_1m_Table_A")
df_left.head(10)

countryname countrycode
0         Germany         DEU
1          France         FRA
2  United Kingdom         GBR
3            Iraq         IRQ
4   United States         USA

In [19]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_1m_Table_B")
df_right.head(10)

countrycode  year  NY_GDP_MKTP_CD
0         ASM  2002        0.514000
1         ASM  2001        0.000000
2         ASM  2003        0.527000
3         BWA  2003        7.511582
4         BWA  2001        5.489608
5         BWA  2002        5.438857
6         DEU  2002     2079.136081
7         DEU  2003     2505.733634
8         DEU  2001     1950.648770
9         FRA  2001     1382.218345

### Inner join

In [20]:
# expecting a 1:1 relation will fail
# df_join = df_left.merge(df_right, how="inner", on="countrycode", validate='1:1')

# explicitly expect a 1:m relation
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:m")
df_join

countryname countrycode  year  NY_GDP_MKTP_CD
0         Germany         DEU  2002     2079.136081
1         Germany         DEU  2003     2505.733634
2         Germany         DEU  2001     1950.648770
3          France         FRA  2001     1382.218345
4          France         FRA  2002     1500.337851
5          France         FRA  2003     1848.124153
6  United Kingdom         GBR  2003     2028.488163
7  United Kingdom         GBR  2001     1613.034403
8  United Kingdom         GBR  2002     1757.571942

### Left join

In [21]:
df_join = df_left.merge(df_right, how="left", on="countrycode", validate="1:m")
df_join

countryname countrycode    year  NY_GDP_MKTP_CD
0          Germany         DEU  2002.0     2079.136081
1          Germany         DEU  2003.0     2505.733634
2          Germany         DEU  2001.0     1950.648770
3           France         FRA  2001.0     1382.218345
4           France         FRA  2002.0     1500.337851
5           France         FRA  2003.0     1848.124153
6   United Kingdom         GBR  2003.0     2028.488163
7   United Kingdom         GBR  2001.0     1613.034403
8   United Kingdom         GBR  2002.0     1757.571942
9             Iraq         IRQ     NaN             NaN
10   United States         USA     NaN             NaN

### Right join

In [22]:
df_join = df_left.merge(df_right, how="right", on="countrycode", validate="1:m")
df_join

countryname countrycode  year  NY_GDP_MKTP_CD
0              NaN         ASM  2002        0.514000
1              NaN         ASM  2001        0.000000
2              NaN         ASM  2003        0.527000
3              NaN         BWA  2003        7.511582
4              NaN         BWA  2001        5.489608
5              NaN         BWA  2002        5.438857
6          Germany         DEU  2002     2079.136081
7          Germany         DEU  2003     2505.733634
8          Germany         DEU  2001     1950.648770
9           France         FRA  2001     1382.218345
10          France         FRA  2002     1500.337851
11          France         FRA  2003     1848.124153
12  United Kingdom         GBR  2003     2028.488163
13  United Kingdom         GBR  2001     1613.034403
14  United Kingdom         GBR  2002     1757.571942

### Outer join

In [23]:
df_join = df_left.merge(
    df_right, how="outer", on="countrycode", validate="1:m", indicator="check_merge"
)
df_join

countryname countrycode    year  NY_GDP_MKTP_CD check_merge
0              NaN         ASM  2002.0        0.514000  right_only
1              NaN         ASM  2001.0        0.000000  right_only
2              NaN         ASM  2003.0        0.527000  right_only
3              NaN         BWA  2003.0        7.511582  right_only
4              NaN         BWA  2001.0        5.489608  right_only
5              NaN         BWA  2002.0        5.438857  right_only
6          Germany         DEU  2002.0     2079.136081        both
7          Germany         DEU  2003.0     2505.733634        both
8          Germany         DEU  2001.0     1950.648770        both
9           France         FRA  2001.0     1382.218345        both
10          France         FRA  2002.0     1500.337851        both
11          France         FRA  2003.0     1848.124153        both
12  United Kingdom         GBR  2003.0     2028.488163        both
13  United Kingdom         GBR  2001.0     1613.034403        both
14  United Kingdom         GBR  2002.0     1757.571942        both
15            Iraq         IRQ     NaN             NaN   left_only
16   United States         USA     NaN             NaN   left_only

In [24]:
# Check unmatched rows (counts)
df_join["check_merge"].value_counts()

check_merge
both          9
right_only    6
left_only     2
Name: count, dtype: int64

## m:m join

### Loading the two tables

In [25]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_mm_Table_A")
df_left.head(10)

countryname countrycode                      region
0         Germany         DEU       Europe & Central Asia
1          France         FRA       Europe & Central Asia
2  United Kingdom         GBR       Europe & Central Asia
3            Iraq         IRQ  Middle East & North Africa
4   United States         USA               North America
5          Canada         CAN               North America
6          Brasil         BRA   Latin America & Caribbean
7       Australia         AUS         East Asia & Pacific

In [26]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_mm_Table_B")
df_right.head(10)

region              datasource  regionalrating
0       Europe & Central Asia  Another Ratings Agency             8.0
1  Middle East & North Africa  Another Ratings Agency             NaN
2               North America  Another Ratings Agency             7.0
3                  South Asia  Another Ratings Agency             7.0
4          Sub-Saharan Africa  Another Ratings Agency             5.0
5         East Asia & Pacific  Another Ratings Agency             NaN
6       Europe & Central Asia    Useful Ratings Group             9.0
7  Middle East & North Africa    Useful Ratings Group             NaN
8               North America    Useful Ratings Group             NaN
9                  South Asia    Useful Ratings Group             6.0

### Outer join

In [27]:
# assuming 1:1 or 1:m would fail
# df_left.merge(df_right, how="outer", on="region", validate='1:1')
# df_left.merge(df_right, how="outer", on="region", validate='1:m')

# instead, validate a m:m relation:
df_join = df_left.merge(df_right, how="outer", on="region", validate="m:m", indicator="check_merge")
df_join

countryname countrycode                      region  \
0        Australia         AUS         East Asia & Pacific   
1        Australia         AUS         East Asia & Pacific   
2          Germany         DEU       Europe & Central Asia   
3          Germany         DEU       Europe & Central Asia   
4           France         FRA       Europe & Central Asia   
5           France         FRA       Europe & Central Asia   
6   United Kingdom         GBR       Europe & Central Asia   
7   United Kingdom         GBR       Europe & Central Asia   
8           Brasil         BRA   Latin America & Caribbean   
9             Iraq         IRQ  Middle East & North Africa   
10            Iraq         IRQ  Middle East & North Africa   
11   United States         USA               North America   
12   United States         USA               North America   
13          Canada         CAN               North America   
14          Canada         CAN               North America   
15             NaN         NaN                  South Asia   
16             NaN         NaN                  South Asia   
17             NaN         NaN          Sub-Saharan Africa   
18             NaN         NaN          Sub-Saharan Africa   

                datasource  regionalrating check_merge  
0   Another Ratings Agency             NaN        both  
1     Useful Ratings Group             5.0        both  
2   Another Ratings Agency             8.0        both  
3     Useful Ratings Group             9.0        both  
4   Another Ratings Agency             8.0        both  
5     Useful Ratings Group             9.0        both  
6   Another Ratings Agency             8.0        both  
7     Useful Ratings Group             9.0        both  
8                      NaN             NaN   left_only  
9   Another Ratings Agency             NaN        both  
10    Useful Ratings Group             NaN        both  
11  Another Ratings Agency             7.0        both  
12    Useful Ratings Group             NaN        both  
13  Another Ratings Agency             7.0        both  
14    Useful Ratings Group             NaN        both  
15  Another Ratings Agency             7.0  right_only  
16    Useful Ratings Group             6.0  right_only  
17  Another Ratings Agency             5.0  right_only  
18    Useful Ratings Group             NaN  right_only

### Drop unmatched rows in a controlled fashion (ending up a 'left' join)

In [28]:
# Note rows not matched and drop them
print(df_join["check_merge"].value_counts())

df_join = df_join[df_join["check_merge"] == "both"]
# the dropna may have created a view on the original DataFrame
print(f"Rows after producing left join ('manually'): {len(df_join)}")
df_join

check_merge
both          14
right_only     4
left_only      1
Name: count, dtype: int64
Rows after producing left join ('manually'): 14


countryname countrycode                      region  \
0        Australia         AUS         East Asia & Pacific   
1        Australia         AUS         East Asia & Pacific   
2          Germany         DEU       Europe & Central Asia   
3          Germany         DEU       Europe & Central Asia   
4           France         FRA       Europe & Central Asia   
5           France         FRA       Europe & Central Asia   
6   United Kingdom         GBR       Europe & Central Asia   
7   United Kingdom         GBR       Europe & Central Asia   
9             Iraq         IRQ  Middle East & North Africa   
10            Iraq         IRQ  Middle East & North Africa   
11   United States         USA               North America   
12   United States         USA               North America   
13          Canada         CAN               North America   
14          Canada         CAN               North America   

                datasource  regionalrating check_merge  
0   Another Ratings Agency             NaN        both  
1     Useful Ratings Group             5.0        both  
2   Another Ratings Agency             8.0        both  
3     Useful Ratings Group             9.0        both  
4   Another Ratings Agency             8.0        both  
5     Useful Ratings Group             9.0        both  
6   Another Ratings Agency             8.0        both  
7     Useful Ratings Group             9.0        both  
9   Another Ratings Agency             NaN        both  
10    Useful Ratings Group             NaN        both  
11  Another Ratings Agency             7.0        both  
12    Useful Ratings Group             NaN        both  
13  Another Ratings Agency             7.0        both  
14    Useful Ratings Group             NaN        both

### Drop NA values in the column of interest

If a data sosurce does not provide a `regionalrating` for a region, we can ignore it

In [29]:
df_join = df_join.dropna(subset=["regionalrating"])
# here, we enforce a completely new DataFrame with copy() to avoid warnings in the next cell
df_join = df_join.copy()

# or:
# df_join = df_join[df_join["regionalrating"].notna()]

df_join

countryname countrycode                 region              datasource  \
1        Australia         AUS    East Asia & Pacific    Useful Ratings Group   
2          Germany         DEU  Europe & Central Asia  Another Ratings Agency   
3          Germany         DEU  Europe & Central Asia    Useful Ratings Group   
4           France         FRA  Europe & Central Asia  Another Ratings Agency   
5           France         FRA  Europe & Central Asia    Useful Ratings Group   
6   United Kingdom         GBR  Europe & Central Asia  Another Ratings Agency   
7   United Kingdom         GBR  Europe & Central Asia    Useful Ratings Group   
11   United States         USA          North America  Another Ratings Agency   
13          Canada         CAN          North America  Another Ratings Agency   

    regionalrating check_merge  
1              5.0        both  
2              8.0        both  
3              9.0        both  
4              8.0        both  
5              9.0        both  
6              8.0        both  
7              9.0        both  
11             7.0        both  
13             7.0        both

### Dropping duplicates (in terms of countries)

As there are two possible `datasource`s for our `regionalrating`, we have created duplicates entries for the countries. We would like to keep only one row per country, preferably with the 'Useful Ratings Group' as `datasource`, but, if it is not available *and* 'Another Ratings Agency' is, then we use the latter.

The method `drop_duplicates()` can be used to drop rows that have duplicate entries according to defined `subset` of columns. By default, the 'first' row of a given duplicate set is retained (`keep`='first').

In other words, the order in the dataset decides which rows are to be kept! Here, we want to sort the dataset, so that (if available), the best datasource is 'on top'.

In [30]:
# ensure the desired order using a helper column

df_join.loc[:, "priority"] = (
    100  # start by initializing a sorting column with something that will be sorted to the end of the table
)
# now ensure that the best datasource is used when available:
df_join.loc[
    (df_join.datasource == "Useful Ratings Group"),
    "priority",
] = 0
# the second best option is the other
df_join.loc[
    (df_join.datasource == "Another Ratings Agency"),
    "priority",
] = 1
# now sort accordingly
df_join = df_join.sort_values("priority")
df_join

countryname countrycode                 region              datasource  \
1        Australia         AUS    East Asia & Pacific    Useful Ratings Group   
3          Germany         DEU  Europe & Central Asia    Useful Ratings Group   
7   United Kingdom         GBR  Europe & Central Asia    Useful Ratings Group   
5           France         FRA  Europe & Central Asia    Useful Ratings Group   
4           France         FRA  Europe & Central Asia  Another Ratings Agency   
2          Germany         DEU  Europe & Central Asia  Another Ratings Agency   
6   United Kingdom         GBR  Europe & Central Asia  Another Ratings Agency   
11   United States         USA          North America  Another Ratings Agency   
13          Canada         CAN          North America  Another Ratings Agency   

    regionalrating check_merge  priority  
1              5.0        both         0  
3              9.0        both         0  
7              9.0        both         0  
5              9.0        both         0  
4              8.0        both         1  
2              8.0        both         1  
6              8.0        both         1  
11             7.0        both         1  
13             7.0        both         1

In [31]:
# finally, drop the duplicates
df_join = df_join.drop_duplicates(subset="countrycode")
df_join

countryname countrycode                 region              datasource  \
1        Australia         AUS    East Asia & Pacific    Useful Ratings Group   
3          Germany         DEU  Europe & Central Asia    Useful Ratings Group   
7   United Kingdom         GBR  Europe & Central Asia    Useful Ratings Group   
5           France         FRA  Europe & Central Asia    Useful Ratings Group   
11   United States         USA          North America  Another Ratings Agency   
13          Canada         CAN          North America  Another Ratings Agency   

    regionalrating check_merge  priority  
1              5.0        both         0  
3              9.0        both         0  
7              9.0        both         0  
5              9.0        both         0  
11             7.0        both         1  
13             7.0        both         1

# Unions - the `pd.concat()` function

## `pd.concat()`

The `pd.concat()` function is a very flexible tool. It can be used for combining datasets both in a `.merge()`-like fashion and for creating "unions", i.e. row-wise appending operations. The most useful feature is its capability of handling **lists of `DataFrame`s** instead of only two at a time!

Also see [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat) and [https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#concatenating-objects](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#concatenating-objects)


In [32]:
# create a DataFrame
df_one = pd.DataFrame({"key": ["A", "B"], "value": [10, 20]})
df_one

key  value
0   A     10
1   B     20

In [33]:
# create another DataFrame
df_two = pd.DataFrame({"key": ["C", "D"], "value": [30, 40]})
df_two

key  value
0   C     30
1   D     40

In [34]:
# let us create yet another DataFrame
df_three = pd.DataFrame({"key": ["E", "F", "G"], "value": [50, 60, 70]})
df_three

key  value
0   E     50
1   F     60
2   G     70

In [35]:
# now concatenate
pd.concat([df_one, df_two, df_three])

key  value
0   A     10
1   B     20
0   C     30
1   D     40
0   E     50
1   F     60
2   G     70

In [36]:
# create a new index
pd.concat([df_one, df_two, df_three], ignore_index=True)

key  value
0   A     10
1   B     20
2   C     30
3   D     40
4   E     50
5   F     60
6   G     70

## Exercise 2

1. Load the first sheet of the Excel file "wdi_reduced.xlsx" into a pandas DataFrame (see [here](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for help with `pandas.read_excel()`)
2. Split the dataset wdi_prepared into two DataFrames (one for years < 2000 and one for years >= 2000).
3. Then concatenate the two dataframes into a new combined dataframe. Check that the original dataframe and the newly created dataframe have the same length.
4. BONUS: split the dataset into one DataFrame for each year and store the DataFrames in a list.
5. BONUS: combine the list datasets using the `pd.concat()` function.